# Importações necessárias

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *

# Iniciando SparkSession

In [2]:
spark = SparkSession.builder \
    .master("local") \
        .appName("bancos-e-tabelas") \
            .config("spark.executer.memory", "1gb") \
                .getOrCreate()

22/06/07 17:03:55 WARN Utils: Your hostname, rbsmotta resolves to a loopback address: 127.0.1.1; using 192.168.1.108 instead (on interface enp1s0)
22/06/07 17:03:55 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/06/07 17:04:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Banco de Dados e Tabelas

In [3]:
# mostrando banco de dados disponiveis
spark.sql("show databases").show()

+---------+
|namespace|
+---------+
|  default|
+---------+



In [5]:
# criando uma nova database
spark.sql("create database desp")

DataFrame[]

In [6]:
# verificar se banco foi criado
spark.sql("show databases").show()

+---------+
|namespace|
+---------+
|  default|
|     desp|
+---------+



In [7]:
# usar banco de dados criado
spark.sql("use desp")

DataFrame[]

In [8]:
# criando schema para dataframe
arqschema = "id INT, nome STRING, status STRING, cidade STRING, vendas INT, data STRING"

In [10]:
# importando dados csv para dataframe
despachantes = spark.read.csv("/home/robson/repositorios/spark/download/despachantes.csv", header=False, schema = arqschema)

In [11]:
despachantes.show()

+---+-------------------+------+-------------+------+----------+
| id|               nome|status|       cidade|vendas|      data|
+---+-------------------+------+-------------+------+----------+
|  1|   Carminda Pestana| Ativo|  Santa Maria|    23|2020-08-11|
|  2|    Deolinda Vilela| Ativo|Novo Hamburgo|    34|2020-03-05|
|  3|   Emídio Dornelles| Ativo| Porto Alegre|    34|2020-02-05|
|  4|Felisbela Dornelles| Ativo| Porto Alegre|    36|2020-02-05|
|  5|     Graça Ornellas| Ativo| Porto Alegre|    12|2020-02-05|
|  6|   Matilde Rebouças| Ativo| Porto Alegre|    22|2019-01-05|
|  7|    Noêmia   Orriça| Ativo|  Santa Maria|    45|2019-10-05|
|  8|      Roque Vásquez| Ativo| Porto Alegre|    65|2020-03-05|
|  9|      Uriel Queiroz| Ativo| Porto Alegre|    54|2018-05-05|
| 10|   Viviana Sequeira| Ativo| Porto Alegre|     0|2020-09-05|
+---+-------------------+------+-------------+------+----------+



In [19]:
# transformando dataframe em uma tabela (daframe "despachantes" dará origem a tabela "Despachantes" 
# no banco de dados "desp")
despachantes.write.saveAsTable("Despachantes")

In [20]:
# para verificar se a tabela foi criada
spark.sql("select * from Despachantes").show()

+---+-------------------+------+-------------+------+----------+
| id|               nome|status|       cidade|vendas|      data|
+---+-------------------+------+-------------+------+----------+
|  1|   Carminda Pestana| Ativo|  Santa Maria|    23|2020-08-11|
|  2|    Deolinda Vilela| Ativo|Novo Hamburgo|    34|2020-03-05|
|  3|   Emídio Dornelles| Ativo| Porto Alegre|    34|2020-02-05|
|  4|Felisbela Dornelles| Ativo| Porto Alegre|    36|2020-02-05|
|  5|     Graça Ornellas| Ativo| Porto Alegre|    12|2020-02-05|
|  6|   Matilde Rebouças| Ativo| Porto Alegre|    22|2019-01-05|
|  7|    Noêmia   Orriça| Ativo|  Santa Maria|    45|2019-10-05|
|  8|      Roque Vásquez| Ativo| Porto Alegre|    65|2020-03-05|
|  9|      Uriel Queiroz| Ativo| Porto Alegre|    54|2018-05-05|
| 10|   Viviana Sequeira| Ativo| Porto Alegre|     0|2020-09-05|
+---+-------------------+------+-------------+------+----------+



In [21]:
# outra forma de verificar
spark.sql("show tables").show()

+---------+------------+-----------+
|namespace|   tableName|isTemporary|
+---------+------------+-----------+
|     desp|despachantes|      false|
+---------+------------+-----------+



In [22]:
# modo overwrite escreve por cima da tabela criada
# modo append adiciona novas linhas de registro
despachantes.write.mode("overwrite").saveAsTable("Despachantes2")

# Tabelas Gerenciadas e Externas

In [23]:
# criando um df de uma tabela existente
despachantes1 = spark.sql("select * from despachantes2")
despachantes1.show()

+---+-------------------+------+-------------+------+----------+
| id|               nome|status|       cidade|vendas|      data|
+---+-------------------+------+-------------+------+----------+
|  1|   Carminda Pestana| Ativo|  Santa Maria|    23|2020-08-11|
|  2|    Deolinda Vilela| Ativo|Novo Hamburgo|    34|2020-03-05|
|  3|   Emídio Dornelles| Ativo| Porto Alegre|    34|2020-02-05|
|  4|Felisbela Dornelles| Ativo| Porto Alegre|    36|2020-02-05|
|  5|     Graça Ornellas| Ativo| Porto Alegre|    12|2020-02-05|
|  6|   Matilde Rebouças| Ativo| Porto Alegre|    22|2019-01-05|
|  7|    Noêmia   Orriça| Ativo|  Santa Maria|    45|2019-10-05|
|  8|      Roque Vásquez| Ativo| Porto Alegre|    65|2020-03-05|
|  9|      Uriel Queiroz| Ativo| Porto Alegre|    54|2018-05-05|
| 10|   Viviana Sequeira| Ativo| Porto Alegre|     0|2020-09-05|
+---+-------------------+------+-------------+------+----------+



In [24]:
# criando arquivo parquet para tabela externa
despachantes.write.format("parquet") \
    .save("/media/robson/HD2/cursos/pyspark/anotacoes_e_praticas/dados/desparquet")

In [25]:
# criando uma tabela externa
despachantes.write.option("path","/media/robson/HD2/cursos/pyspark/anotacoes_e_praticas/dados/desparquet/externa") \
    .saveAsTable("Despachantes_ng")

In [26]:
# testando se a tabela foi criada
spark.sql("select * from Despachantes_ng").show()

+---+-------------------+------+-------------+------+----------+
| id|               nome|status|       cidade|vendas|      data|
+---+-------------------+------+-------------+------+----------+
|  1|   Carminda Pestana| Ativo|  Santa Maria|    23|2020-08-11|
|  2|    Deolinda Vilela| Ativo|Novo Hamburgo|    34|2020-03-05|
|  3|   Emídio Dornelles| Ativo| Porto Alegre|    34|2020-02-05|
|  4|Felisbela Dornelles| Ativo| Porto Alegre|    36|2020-02-05|
|  5|     Graça Ornellas| Ativo| Porto Alegre|    12|2020-02-05|
|  6|   Matilde Rebouças| Ativo| Porto Alegre|    22|2019-01-05|
|  7|    Noêmia   Orriça| Ativo|  Santa Maria|    45|2019-10-05|
|  8|      Roque Vásquez| Ativo| Porto Alegre|    65|2020-03-05|
|  9|      Uriel Queiroz| Ativo| Porto Alegre|    54|2018-05-05|
| 10|   Viviana Sequeira| Ativo| Porto Alegre|     0|2020-09-05|
+---+-------------------+------+-------------+------+----------+



In [27]:
spark.sql("show tables").show()

+---------+---------------+-----------+
|namespace|      tableName|isTemporary|
+---------+---------------+-----------+
|     desp|   despachantes|      false|
|     desp|  despachantes2|      false|
|     desp|despachantes_ng|      false|
+---------+---------------+-----------+



In [28]:
# verificando se a tabela é gerenciada ou não
spark.sql("show create table Despachantes") \
    .show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------+
|createtab_stmt                                                                                                                                                 |
+---------------------------------------------------------------------------------------------------------------------------------------------------------------+
|CREATE TABLE `desp`.`Despachantes` (\n  `id` INT,\n  `nome` STRING,\n  `status` STRING,\n  `cidade` STRING,\n  `vendas` INT,\n  `data` STRING)\nUSING parquet\n|
+---------------------------------------------------------------------------------------------------------------------------------------------------------------+



In [29]:
# verificando se a tabela é gerenciada ou não
spark.sql("show create table Despachantes_ng") \
    .show(truncate=False)

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|createtab_stmt                                                                                                                                                                                                                                                    |
+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|CREATE TABLE `desp`.`Despachantes_ng` (\n  `id` INT,\n  `nome` STRING,\n  `status` STRING,\n  `cidade` STRING,\n  `vendas` INT,\n  `data` STRING)\nUSING parquet\nLOCATION 'file:/media/robson/HD2/cursos/pyspark/anotac

In [30]:
# listando tabelas
spark.catalog.listTables()

[Table(name='despachantes', database='desp', description=None, tableType='MANAGED', isTemporary=False),
 Table(name='despachantes2', database='desp', description=None, tableType='MANAGED', isTemporary=False),
 Table(name='despachantes_ng', database='desp', description=None, tableType='EXTERNAL', isTemporary=False)]

# Views

In [31]:
# criando view temporária utilizando api de dataframe
despachantes.createOrReplaceTempView("Despachantes_view1")

In [32]:
# consultando view
spark.sql("select * from Despachantes_view1") \
    .show()

+---+-------------------+------+-------------+------+----------+
| id|               nome|status|       cidade|vendas|      data|
+---+-------------------+------+-------------+------+----------+
|  1|   Carminda Pestana| Ativo|  Santa Maria|    23|2020-08-11|
|  2|    Deolinda Vilela| Ativo|Novo Hamburgo|    34|2020-03-05|
|  3|   Emídio Dornelles| Ativo| Porto Alegre|    34|2020-02-05|
|  4|Felisbela Dornelles| Ativo| Porto Alegre|    36|2020-02-05|
|  5|     Graça Ornellas| Ativo| Porto Alegre|    12|2020-02-05|
|  6|   Matilde Rebouças| Ativo| Porto Alegre|    22|2019-01-05|
|  7|    Noêmia   Orriça| Ativo|  Santa Maria|    45|2019-10-05|
|  8|      Roque Vásquez| Ativo| Porto Alegre|    65|2020-03-05|
|  9|      Uriel Queiroz| Ativo| Porto Alegre|    54|2018-05-05|
| 10|   Viviana Sequeira| Ativo| Porto Alegre|     0|2020-09-05|
+---+-------------------+------+-------------+------+----------+



In [33]:
# criando uma view global
despachantes.createGlobalTempView("Despachantes_view2")

In [34]:
# consultando view global
spark.sql("select * from global_temp.Despachantes_view2") \
    .show()

+---+-------------------+------+-------------+------+----------+
| id|               nome|status|       cidade|vendas|      data|
+---+-------------------+------+-------------+------+----------+
|  1|   Carminda Pestana| Ativo|  Santa Maria|    23|2020-08-11|
|  2|    Deolinda Vilela| Ativo|Novo Hamburgo|    34|2020-03-05|
|  3|   Emídio Dornelles| Ativo| Porto Alegre|    34|2020-02-05|
|  4|Felisbela Dornelles| Ativo| Porto Alegre|    36|2020-02-05|
|  5|     Graça Ornellas| Ativo| Porto Alegre|    12|2020-02-05|
|  6|   Matilde Rebouças| Ativo| Porto Alegre|    22|2019-01-05|
|  7|    Noêmia   Orriça| Ativo|  Santa Maria|    45|2019-10-05|
|  8|      Roque Vásquez| Ativo| Porto Alegre|    65|2020-03-05|
|  9|      Uriel Queiroz| Ativo| Porto Alegre|    54|2018-05-05|
| 10|   Viviana Sequeira| Ativo| Porto Alegre|     0|2020-09-05|
+---+-------------------+------+-------------+------+----------+



In [35]:
# outra forma de criar views temporarias (comando SQL)
spark.sql("CREATE OR REPLACE TEMP VIEW DESP_VIEW AS select * from despachantes")

DataFrame[]

In [36]:
# consultando view
spark.sql("select * from DESP_VIEW").show()

+---+-------------------+------+-------------+------+----------+
| id|               nome|status|       cidade|vendas|      data|
+---+-------------------+------+-------------+------+----------+
|  1|   Carminda Pestana| Ativo|  Santa Maria|    23|2020-08-11|
|  2|    Deolinda Vilela| Ativo|Novo Hamburgo|    34|2020-03-05|
|  3|   Emídio Dornelles| Ativo| Porto Alegre|    34|2020-02-05|
|  4|Felisbela Dornelles| Ativo| Porto Alegre|    36|2020-02-05|
|  5|     Graça Ornellas| Ativo| Porto Alegre|    12|2020-02-05|
|  6|   Matilde Rebouças| Ativo| Porto Alegre|    22|2019-01-05|
|  7|    Noêmia   Orriça| Ativo|  Santa Maria|    45|2019-10-05|
|  8|      Roque Vásquez| Ativo| Porto Alegre|    65|2020-03-05|
|  9|      Uriel Queiroz| Ativo| Porto Alegre|    54|2018-05-05|
| 10|   Viviana Sequeira| Ativo| Porto Alegre|     0|2020-09-05|
+---+-------------------+------+-------------+------+----------+



In [37]:
# outra forma de criar views globais (comando SQL)
spark.sql("CREATE OR REPLACE GLOBAL TEMP VIEW DESP_VIEW AS select * from despachantes")

DataFrame[]

In [38]:
# para consultar
spark.sql("select * from global_temp.DESP_VIEW").show()

+---+-------------------+------+-------------+------+----------+
| id|               nome|status|       cidade|vendas|      data|
+---+-------------------+------+-------------+------+----------+
|  1|   Carminda Pestana| Ativo|  Santa Maria|    23|2020-08-11|
|  2|    Deolinda Vilela| Ativo|Novo Hamburgo|    34|2020-03-05|
|  3|   Emídio Dornelles| Ativo| Porto Alegre|    34|2020-02-05|
|  4|Felisbela Dornelles| Ativo| Porto Alegre|    36|2020-02-05|
|  5|     Graça Ornellas| Ativo| Porto Alegre|    12|2020-02-05|
|  6|   Matilde Rebouças| Ativo| Porto Alegre|    22|2019-01-05|
|  7|    Noêmia   Orriça| Ativo|  Santa Maria|    45|2019-10-05|
|  8|      Roque Vásquez| Ativo| Porto Alegre|    65|2020-03-05|
|  9|      Uriel Queiroz| Ativo| Porto Alegre|    54|2018-05-05|
| 10|   Viviana Sequeira| Ativo| Porto Alegre|     0|2020-09-05|
+---+-------------------+------+-------------+------+----------+



# Comparando DataFrames com Tabelas

In [39]:
# importando modulos
from pyspark.sql import functions as Func
from pyspark.sql.functions import *

In [40]:

spark.sql("select * from Despachantes2") \
    .show()

+---+-------------------+------+-------------+------+----------+
| id|               nome|status|       cidade|vendas|      data|
+---+-------------------+------+-------------+------+----------+
|  1|   Carminda Pestana| Ativo|  Santa Maria|    23|2020-08-11|
|  2|    Deolinda Vilela| Ativo|Novo Hamburgo|    34|2020-03-05|
|  3|   Emídio Dornelles| Ativo| Porto Alegre|    34|2020-02-05|
|  4|Felisbela Dornelles| Ativo| Porto Alegre|    36|2020-02-05|
|  5|     Graça Ornellas| Ativo| Porto Alegre|    12|2020-02-05|
|  6|   Matilde Rebouças| Ativo| Porto Alegre|    22|2019-01-05|
|  7|    Noêmia   Orriça| Ativo|  Santa Maria|    45|2019-10-05|
|  8|      Roque Vásquez| Ativo| Porto Alegre|    65|2020-03-05|
|  9|      Uriel Queiroz| Ativo| Porto Alegre|    54|2018-05-05|
| 10|   Viviana Sequeira| Ativo| Porto Alegre|     0|2020-09-05|
+---+-------------------+------+-------------+------+----------+



In [41]:
# limitar consulta a certas colunas
spark.sql("select nome, vendas from Despachantes2") \
    .show()

+-------------------+------+
|               nome|vendas|
+-------------------+------+
|   Carminda Pestana|    23|
|    Deolinda Vilela|    34|
|   Emídio Dornelles|    34|
|Felisbela Dornelles|    36|
|     Graça Ornellas|    12|
|   Matilde Rebouças|    22|
|    Noêmia   Orriça|    45|
|      Roque Vásquez|    65|
|      Uriel Queiroz|    54|
|   Viviana Sequeira|     0|
+-------------------+------+



In [42]:
despachantes.select("nome", "vendas") \
    .show()

+-------------------+------+
|               nome|vendas|
+-------------------+------+
|   Carminda Pestana|    23|
|    Deolinda Vilela|    34|
|   Emídio Dornelles|    34|
|Felisbela Dornelles|    36|
|     Graça Ornellas|    12|
|   Matilde Rebouças|    22|
|    Noêmia   Orriça|    45|
|      Roque Vásquez|    65|
|      Uriel Queiroz|    54|
|   Viviana Sequeira|     0|
+-------------------+------+



In [43]:
# utilizando clausula com o select do SQL
spark.sql("select nome, vendas from Despachantes where vendas > 20") \
    .show()

+-------------------+------+
|               nome|vendas|
+-------------------+------+
|   Carminda Pestana|    23|
|    Deolinda Vilela|    34|
|   Emídio Dornelles|    34|
|Felisbela Dornelles|    36|
|   Matilde Rebouças|    22|
|    Noêmia   Orriça|    45|
|      Roque Vásquez|    65|
|      Uriel Queiroz|    54|
+-------------------+------+



In [44]:
despachantes.select("nome", "vendas") \
    .where(Func.col("vendas") > 20) \
        .show()

+-------------------+------+
|               nome|vendas|
+-------------------+------+
|   Carminda Pestana|    23|
|    Deolinda Vilela|    34|
|   Emídio Dornelles|    34|
|Felisbela Dornelles|    36|
|   Matilde Rebouças|    22|
|    Noêmia   Orriça|    45|
|      Roque Vásquez|    65|
|      Uriel Queiroz|    54|
+-------------------+------+



In [45]:
# vendas por cidade ordenado de forma decrescente 
spark.sql("select cidade, sum(vendas) from Despachantes group by cidade order by 2 desc") \
    .show()

+-------------+-----------+
|       cidade|sum(vendas)|
+-------------+-----------+
| Porto Alegre|        223|
|  Santa Maria|         68|
|Novo Hamburgo|         34|
+-------------+-----------+



In [46]:
# mesma coisa, com a api DataFrame
despachantes.groupBy("cidade") \
    .agg(sum("vendas")) \
        .orderBy(Func.col("sum(vendas)") \
            .desc()).show()

+-------------+-----------+
|       cidade|sum(vendas)|
+-------------+-----------+
| Porto Alegre|        223|
|  Santa Maria|         68|
|Novo Hamburgo|         34|
+-------------+-----------+



# Joins com DataFrames e SQL

In [47]:
# criando schema do dataframe reclamacoes
recschema = "idrec INT, datarec STRING, iddesp INT"

In [49]:
# criando dataframe de um arquivo csv
reclamacoes = spark.read.csv("/home/robson/repositorios/spark/download/reclamacoes.csv", header=False, schema=recschema)

In [50]:
reclamacoes.show()

+-----+----------+------+
|idrec|   datarec|iddesp|
+-----+----------+------+
|    1|2020-09-12|     2|
|    2|2020-09-11|     2|
|    3|2020-10-05|     4|
|    4|2020-10-02|     5|
|    5|2020-12-06|     5|
|    6|2020-01-09|     5|
|    7|2020-01-05|     9|
+-----+----------+------+



In [51]:
# criando tabela no banco de dados
reclamacoes.write.saveAsTable("reclamacoes")

In [52]:
# fazendo o inner join
spark.sql("select reclamacoes.*, despachantes.nome from despachantes \
    inner join reclamacoes on (despachantes.id = reclamacoes.iddesp)") \
        .show()

+-----+----------+------+-------------------+
|idrec|   datarec|iddesp|               nome|
+-----+----------+------+-------------------+
|    1|2020-09-12|     2|    Deolinda Vilela|
|    2|2020-09-11|     2|    Deolinda Vilela|
|    3|2020-10-05|     4|Felisbela Dornelles|
|    4|2020-10-02|     5|     Graça Ornellas|
|    5|2020-12-06|     5|     Graça Ornellas|
|    6|2020-01-09|     5|     Graça Ornellas|
|    7|2020-01-05|     9|      Uriel Queiroz|
+-----+----------+------+-------------------+



In [53]:
# utilizando o right join
spark.sql("select reclamacoes.*, despachantes.nome from despachantes \
    right join reclamacoes on (despachantes.id = reclamacoes.iddesp)") \
        .show()

+-----+----------+------+-------------------+
|idrec|   datarec|iddesp|               nome|
+-----+----------+------+-------------------+
|    1|2020-09-12|     2|    Deolinda Vilela|
|    2|2020-09-11|     2|    Deolinda Vilela|
|    3|2020-10-05|     4|Felisbela Dornelles|
|    4|2020-10-02|     5|     Graça Ornellas|
|    5|2020-12-06|     5|     Graça Ornellas|
|    6|2020-01-09|     5|     Graça Ornellas|
|    7|2020-01-05|     9|      Uriel Queiroz|
+-----+----------+------+-------------------+



In [54]:
# utilizando left join (no caso aparecem despachantes que NÂO tiveram reclamacoes)
spark.sql("select reclamacoes.*, despachantes.nome from despachantes \
    left join reclamacoes on (despachantes.id = reclamacoes.iddesp)") \
        .show()

+-----+----------+------+-------------------+
|idrec|   datarec|iddesp|               nome|
+-----+----------+------+-------------------+
| null|      null|  null|   Carminda Pestana|
|    2|2020-09-11|     2|    Deolinda Vilela|
|    1|2020-09-12|     2|    Deolinda Vilela|
| null|      null|  null|   Emídio Dornelles|
|    3|2020-10-05|     4|Felisbela Dornelles|
|    6|2020-01-09|     5|     Graça Ornellas|
|    5|2020-12-06|     5|     Graça Ornellas|
|    4|2020-10-02|     5|     Graça Ornellas|
| null|      null|  null|   Matilde Rebouças|
| null|      null|  null|    Noêmia   Orriça|
| null|      null|  null|      Roque Vásquez|
|    7|2020-01-05|     9|      Uriel Queiroz|
| null|      null|  null|   Viviana Sequeira|
+-----+----------+------+-------------------+



In [55]:
# juncoes utilizando inner com api de dataframes
despachantes.join(reclamacoes, despachantes.id == reclamacoes.iddesp, "inner") \
    .select("idrec", "datarec", "iddesp", "nome") \
        .show()

+-----+----------+------+-------------------+
|idrec|   datarec|iddesp|               nome|
+-----+----------+------+-------------------+
|    2|2020-09-11|     2|    Deolinda Vilela|
|    1|2020-09-12|     2|    Deolinda Vilela|
|    3|2020-10-05|     4|Felisbela Dornelles|
|    6|2020-01-09|     5|     Graça Ornellas|
|    5|2020-12-06|     5|     Graça Ornellas|
|    4|2020-10-02|     5|     Graça Ornellas|
|    7|2020-01-05|     9|      Uriel Queiroz|
+-----+----------+------+-------------------+



In [56]:
# juncoes utilizando right com api de dataframes
despachantes.join(reclamacoes, despachantes.id == reclamacoes.iddesp, "right") \
    .select("idrec", "datarec", "iddesp", "nome") \
        .show()

+-----+----------+------+-------------------+
|idrec|   datarec|iddesp|               nome|
+-----+----------+------+-------------------+
|    1|2020-09-12|     2|    Deolinda Vilela|
|    2|2020-09-11|     2|    Deolinda Vilela|
|    3|2020-10-05|     4|Felisbela Dornelles|
|    4|2020-10-02|     5|     Graça Ornellas|
|    5|2020-12-06|     5|     Graça Ornellas|
|    6|2020-01-09|     5|     Graça Ornellas|
|    7|2020-01-05|     9|      Uriel Queiroz|
+-----+----------+------+-------------------+



In [57]:
# juncoes utilizando left com api de dataframes (aparecem despachantes sem reclamacoes)
despachantes.join(reclamacoes, despachantes.id == reclamacoes.iddesp, "left") \
    .select("idrec", "datarec", "iddesp", "nome") \
        .show()

+-----+----------+------+-------------------+
|idrec|   datarec|iddesp|               nome|
+-----+----------+------+-------------------+
| null|      null|  null|   Carminda Pestana|
|    2|2020-09-11|     2|    Deolinda Vilela|
|    1|2020-09-12|     2|    Deolinda Vilela|
| null|      null|  null|   Emídio Dornelles|
|    3|2020-10-05|     4|Felisbela Dornelles|
|    6|2020-01-09|     5|     Graça Ornellas|
|    5|2020-12-06|     5|     Graça Ornellas|
|    4|2020-10-02|     5|     Graça Ornellas|
| null|      null|  null|   Matilde Rebouças|
| null|      null|  null|    Noêmia   Orriça|
| null|      null|  null|      Roque Vásquez|
|    7|2020-01-05|     9|      Uriel Queiroz|
| null|      null|  null|   Viviana Sequeira|
+-----+----------+------+-------------------+



# Utilizando Spark-SQL

In [58]:
spark.sql('use desp')

DataFrame[]

In [59]:
spark.sql("show tables").show()


+---------+------------------+-----------+
|namespace|         tableName|isTemporary|
+---------+------------------+-----------+
|     desp|      despachantes|      false|
|     desp|     despachantes2|      false|
|     desp|   despachantes_ng|      false|
|     desp|       reclamacoes|      false|
|         |         desp_view|       true|
|         |despachantes_view1|       true|
+---------+------------------+-----------+

